In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import (
    confusion_matrix,
    ConfusionMatrixDisplay,
    roc_curve,
    auc,
    classification_report,
)
from sklearn.preprocessing import StandardScaler
from matplotlib import colormaps
import numpy as np
from sklearn.pipeline import Pipeline
import random
from matplotlib import pyplot as plt

In [ ]:
training_file = "wildfires_training.csv"
test_file = "wildfires_test.csv"

df = pd.read_csv("wildfires_training.csv")
df_test = pd.read_csv("wildfires_test.csv")

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.shape

## Pre-Processing

In [ ]:
# Interaction Months
df["temp_humidity"] = df["temp"] * df["humidity"]
df["wind_bui"] = df["wind_speed"] * df["buildup_index"]

df_test["temp_humidity"] = df_test["temp"] * df_test["humidity"]
df_test["wind_bui"] = df_test["wind_speed"] * df_test["buildup_index"]

In [ ]:
dependent_variable = "fire"
independent_variables = [col for col in df.columns if col != dependent_variable]

In [ ]:
# Training predictors and response variables
X_training = df[independent_variables]
y_training = df[dependent_variable] == "yes"

In [ ]:
X_test = df_test[independent_variables]
y_test = df_test[dependent_variable] == "yes"

In [ ]:
# Time to re-run with Hyper-parameter tuning
penalties = ["l1", "l2", "elasticnet", None]
c_values = np.logspace(-3, 3, 100)

In [ ]:
training_test_comparison = {}

for penalty in penalties:
    models_info = []
    for c in c_values:
        l1_ratio = 0.3 if penalty == "elasticnet" else None

        if penalty == "l1":
            solver = "liblinear"
        elif penalty == "elasticnet":
            solver = "saga"
        else:
            solver = "lbfgs"

        logreg_pipe = Pipeline(
            [
                ("scaler", StandardScaler()),
                (
                    "logreg",
                    LogisticRegression(
                        random_state=42,
                        penalty=penalty,
                        solver=solver,
                        max_iter=5000,
                        C=c if penalty is not None else 1.0,
                        l1_ratio=l1_ratio,
                    ),
                ),
            ]
        )

        logreg_pipe.fit(X_training, y_training)
        predictions_training = logreg_pipe.predict(X_training)
        predictions_test = logreg_pipe.predict(X_test)

        accuracy_training = metrics.accuracy_score(y_training, predictions_training)
        accuracy_test = metrics.accuracy_score(y_test, predictions_test)

        y_probs = logreg_pipe.predict_proba(X_test)[:, 1]

        models_info.append(
            {
                "C": c,
                "training_accuracy": accuracy_training,
                "test_accuracy": accuracy_test,
                "pipeline": logreg_pipe,
                "predictions_test": predictions_test,
            }
        )

    # Save all models for this penalty
    training_test_comparison[penalty] = models_info

In [ ]:
best_model = {}
for pen in penalties:
    best_model[pen] = max(
        training_test_comparison[pen], key=lambda x: x["test_accuracy"]
    )

best_model

In [ ]:
# Print nicely
for penalty, model in best_model.items():
    print(f"Best {penalty if penalty else '(Without Regularization)'} model:")
    print(f"  C = {model['C']}")
    print(f"  Training Accuracy = {model['training_accuracy']:.4f}")
    print(f"  Test Accuracy = {model['test_accuracy']:.4f}\n")

## Comparison Graphs
Below are graphs showing the effect of penalty functions and C Values (Lambda Inverse) on Logistic Regression

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(6, 6))
ax = ax.flatten()

for i, (penalty, model_info) in enumerate(best_model.items()):
    y_pred = model_info["predictions_test"]

    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["No", "Yes"])
    disp.plot(cmap=random.choice(list(colormaps)), ax=ax[i], colorbar=False)
    disp.ax_.set_title(f"Penalty: {penalty}")

plt.tight_layout()
plt.show()

### Training and Test Set Accuracy

In [ ]:
l1 = training_test_comparison["l1"]
accuracy_training_l1 = [el["training_accuracy"] for el in l1]
accuracy_test_l1 = [el["test_accuracy"] for el in l1]

#### L1 Logistic Regression

In [ ]:
plt.scatter(c_values, accuracy_training_l1, marker="x")
plt.scatter(c_values, accuracy_test_l1, marker="+")
plt.xscale("log")  # use log scale for C
plt.ylim([0.0, 1.1])
plt.xlabel("Value of C (log scale)")
plt.ylabel("Accuracy")
plt.legend(["Training", "Test"], loc=4)
plt.title(
    "Effect of C on training and Test set accuracy on L1 Regularization", fontsize=10
)
plt.show()

#### L2 Logistic Regression

In [ ]:
l2 = training_test_comparison["l2"]
accuracy_training_l2 = [el["training_accuracy"] for el in l2]
accuracy_test_l2 = [el["test_accuracy"] for el in l2]

plt.scatter(c_values, accuracy_training_l2, marker="x")
plt.scatter(c_values, accuracy_test_l2, marker="+")
plt.xscale("log")  # use log scale for C
plt.ylim([0.0, 1.1])
plt.xlabel("Value of C (log scale)")
plt.ylabel("Accuracy")
plt.legend(["Training", "Test"], loc=4)
plt.title(
    "Effect of C on training and Test set accuracy on L2 Regularization", fontsize=10
)
plt.show()

#### ElasticNet Logistic Regression

In [ ]:
elasticnet = training_test_comparison["elasticnet"]
accuracy_training_elasticnet = [el["training_accuracy"] for el in elasticnet]
accuracy_test_elasticnet = [el["test_accuracy"] for el in elasticnet]

plt.scatter(c_values, accuracy_training_elasticnet, marker="x")
plt.scatter(c_values, accuracy_test_elasticnet, marker="+")
plt.xscale("log")  # use log scale for C
plt.ylim([0.0, 1.1])
plt.xlabel("Value of C (log scale)")
plt.ylabel("Accuracy")
plt.legend(["Training", "Test"], loc=4)
plt.title("Effect of C on training and Test set accuracy on ElasticNet", fontsize=10)
plt.show()

### Coefficient Magnitudes Vs C

Elasticnet Coefficient Magnitude

In [ ]:
coef_norms_elasticnet = [np.sum(np.abs(el["coefficients"])) for el in elasticnet]
plt.plot(c_values, coef_norms_elasticnet, marker="x")
plt.xscale("log")
plt.xlabel("C (log scale)")
plt.ylabel("Sum of |coefficients|")
plt.title("Elasticnet Coefficient Magnitude vs C")
plt.show()

#### L1 Logistic Regression

In [ ]:
coef_norms_l1 = [np.sum(np.abs(el["coefficients"])) for el in l1]
plt.plot(c_values, coef_norms_l1, marker="x")
plt.xscale("log")
plt.xlabel("C (log scale)")
plt.ylabel("Sum of |coefficients|")
plt.title("L1 Coefficient Magnitude vs C")
plt.show()

#### L2 Logistic Regression

In [ ]:
coef_norms_l2 = [np.sum(np.abs(el["coefficients"])) for el in l2]
plt.plot(c_values, coef_norms_l2, marker="x")
plt.xscale("log")
plt.xlabel("C (log scale)")
plt.ylabel("Sum of |coefficients|")
plt.title("L1 Coefficient Magnitude vs C")
plt.show()

In [ ]:
num_nonzero_elasticnet = [np.count_nonzero(el["coefficients"]) for el in elasticnet]
plt.plot(c_values, num_nonzero_elasticnet, marker="o")
plt.xscale("log")
plt.xlabel("C (log scale)")
plt.ylabel("Number of Nonzero Coefficients")
plt.title("L1 Sparsity vs C")
plt.show()

### Coefficient Sparsity

#### L2 Logistic Regression

In [ ]:
num_nonzero_elasticnet = [np.count_nonzero(el["coefficients"]) for el in elasticnet]
plt.plot(c_values, num_nonzero_elasticnet, marker="o")
plt.xscale("log")
plt.xlabel("C (log scale)")
plt.ylabel("Number of Nonzero Coefficients")
plt.title("ElasticNet Sparsity vs C")
plt.show()

### ROC Curve

In [ ]:
for i, pen in enumerate(["l1", "l2", "elasticnet", None]):
    best_model = max(training_test_comparison[pen], key=lambda x: x["test_accuracy"])
    y_probs = best_model["pipeline"].predict_proba(X_test)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, y_probs)
    plt.plot(fpr, tpr, label=f"{pen} - AUC {auc(fpr, tpr)}")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.legend(loc="lower right")
    plt.title("ROC Curve")

In [ ]:
for i, pen in enumerate(["l1", "l2", "elasticnet", None]):
    best_model = max(training_test_comparison[pen], key=lambda x: x["test_accuracy"])
    title = f"\n\t\t\t{pen.upper() if type(pen) == str else 'No Regularization'} Classification"
    print(f"{title}\n{classification_report(y_test, best_model['predictions_test'])}")